In [1]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import threading
import time

class FaceEmotionGUI:
    def __init__(self, root):
        # Inisialisasi jendela utama aplikasi
        self.root = root
        self.root.title("Sistem Pengenalan Emosi Wajah Realtime")
        self.root.geometry("1400x800")
        self.root.configure(bg='#f0f0f0')
        
        # Memuat model CNN untuk prediksi emosi dan face cascade untuk deteksi wajah
        self.model = load_model('E:/Pengenalan Wajah CNN K5/best_model.keras')
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.class_labels = ['angry', 'happy', 'neutral', 'sad', 'surprise']
        
        # Variabel untuk kontrol kamera
        self.camera = None
        self.is_camera_active = False
        
        # Memanggil fungsi untuk menyiapkan GUI
        self.setup_gui()
        
    def setup_gui(self):
        # Membuat frame utama untuk membungkus elemen GUI
        main_frame = ttk.Frame(self.root)
        main_frame.pack(padx=20, pady=20, fill='both', expand=True)
        
        # Frame kiri untuk area kamera dan upload gambar
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side='left', padx=10, fill='both', expand=True)
        
        # Membuat frame untuk tombol kontrol
        control_frame = ttk.Frame(left_frame)
        control_frame.pack(pady=10)
        
        # Tombol untuk upload gambar
        upload_btn = ttk.Button(control_frame, text="Upload Gambar", command=self.upload_image)
        upload_btn.pack(side='left', padx=5)
        
        # Tombol untuk mengontrol kamera
        self.camera_btn = ttk.Button(control_frame, text="Mulai Kamera", command=self.toggle_camera)
        self.camera_btn.pack(side='left', padx=5)
        
        # Label untuk preview area (kamera atau gambar)
        self.preview_label = ttk.Label(left_frame)
        self.preview_label.pack(pady=10)
        
        # Frame kanan untuk area hasil prediksi dan grafik
        right_frame = ttk.Frame(main_frame)
        right_frame.pack(side='right', padx=10, fill='both', expand=True)
        
        # Area untuk menampilkan grafik hasil prediksi
        self.fig = Figure(figsize=(6, 4))
        self.canvas = FigureCanvasTkAgg(self.fig, right_frame)
        self.canvas.get_tk_widget().pack(pady=10)
        
        # Label untuk menampilkan hasil prediksi
        self.result_label = ttk.Label(right_frame, text="", font=('Arial', 12))
        self.result_label.pack(pady=10)
        
    def toggle_camera(self):
        # Mengontrol status kamera (aktif atau berhenti)
        if self.is_camera_active:
            self.stop_camera()
            self.camera_btn.configure(text="Mulai Kamera")
        else:
            self.start_camera()
            self.camera_btn.configure(text="Stop Kamera")
        
    def start_camera(self):
        # Memulai kamera menggunakan OpenCV
        self.camera = cv2.VideoCapture(0)
        self.is_camera_active = True
        self.update_camera()
        
    def stop_camera(self):
        # Menghentikan kamera
        self.is_camera_active = False
        if self.camera is not None:
            self.camera.release()
            
    def update_camera(self):
        # Menangkap frame dari kamera secara realtime
        if self.is_camera_active:
            ret, frame = self.camera.read()
            if ret:
                # Deteksi wajah pada frame menggunakan face cascade
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
                
                for (x, y, w, h) in faces:
                    # Menggambar kotak di sekitar wajah
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    
                    # Memproses area wajah untuk prediksi
                    face_gray = gray[y:y+h, x:x+w]
                    face_gray = cv2.resize(face_gray, (48, 48))
                    
                    # Normalisasi dan reshape gambar untuk input model
                    processed_image = face_gray.reshape(1, 48, 48, 1) / 255.0
                    predictions = self.model.predict(processed_image)[0]
                    
                    # Update grafik dan label hasil
                    self.update_plot(predictions)
                    
                    # Menampilkan hasil prediksi pada frame
                    emotion = self.class_labels[np.argmax(predictions)]
                    confidence = np.max(predictions) * 100
                    cv2.putText(frame, f"{emotion} ({confidence:.1f}%)", 
                              (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, 
                              (36,255,12), 2)
                
                # Konversi frame untuk ditampilkan di GUI
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (640, 480))
                photo = ImageTk.PhotoImage(image=Image.fromarray(frame))
                
                self.preview_label.configure(image=photo)
                self.preview_label.image = photo
                
            # Memanggil fungsi ini secara berulang setiap 10ms
            self.root.after(10, self.update_camera)
            
    def upload_image(self):
        # Membuka file dialog untuk memilih gambar
        file_path = filedialog.askopenfilename()
        if file_path:
            # Menghentikan kamera jika sedang aktif
            if self.is_camera_active:
                self.stop_camera()
                self.camera_btn.configure(text="Mulai Kamera")
            
            # Membaca gambar yang diupload
            image = cv2.imread(file_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
            
            for (x, y, w, h) in faces:
                # Memproses area wajah pada gambar untuk prediksi
                face_gray = gray[y:y+h, x:x+w]
                face_gray = cv2.resize(face_gray, (48, 48))
                
                # Prediksi emosi menggunakan model
                processed_image = face_gray.reshape(1, 48, 48, 1) / 255.0
                predictions = self.model.predict(processed_image)[0]
                
                # Update grafik hasil prediksi
                self.update_plot(predictions)
                
                # Menampilkan hasil prediksi pada gambar
                emotion = self.class_labels[np.argmax(predictions)]
                confidence = np.max(predictions) * 100
                cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(image, f"{emotion} ({confidence:.1f}%)", 
                          (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, 
                          (36,255,12), 2)
            
            # Konversi gambar untuk ditampilkan di GUI
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (640, 480))
            photo = ImageTk.PhotoImage(image=Image.fromarray(image))
            self.preview_label.configure(image=photo)
            self.preview_label.image = photo
            
    def update_plot(self, predictions):
        # Membersihkan grafik sebelumnya
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        
        # Membuat bar plot untuk hasil prediksi
        bars = ax.bar(self.class_labels, predictions * 100)
        
        # Menyesuaikan tampilan grafik
        ax.set_ylabel('Confidence (%)')
        ax.set_title('Prediksi Emosi')
        plt.setp(ax.get_xticklabels(), rotation=45)
        
        # Menampilkan nilai confidence di atas tiap bar
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%', ha='center', va='bottom')
        
        # Menyempurnakan tata letak grafik
        self.fig.tight_layout()
        self.canvas.draw()

# Menjalankan aplikasi jika file dieksekusi secara langsung
if __name__ == "__main__":
    root = tk.Tk()
    app = FaceEmotionGUI(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━